In [8]:
# Importation des bibliothèques
import matplotlib.pyplot as plt
import matplotlib.style as style
import matplotlib.gridspec as gridspec
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats

from math import *

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [6]:
def getTrainStationByPosition(latt, long, departement, type_request):
    
    g = rdflib.Graph()
    g.namespace_manager.bind('rdf', Namespace('http://www.w3.org/1999/02/22-rdf-syntax-ns#'))
    g.namespace_manager.bind('rdfs', Namespace('http://www.w3.org/2000/01/rdf-schema#'))
    g.namespace_manager.bind('owl', Namespace('http://www.w3.org/2002/07/owl#'))
    g.namespace_manager.bind('tgv', Namespace('http://www.owl-ontologies.com/unnamed.owl#'))
    g.parse("data/gares-tgv/gares.owl")
    query = 'SELECT DISTINCT ?lat ?long ?nomGare  WHERE { ?gare tgv:Latitude ?lat . ?gare tgv:Longitude ?long . ?gare tgv:Nom_Gare ?nomGare }'
    res = g.query(query)
    
    response = []
    euclid = []
    
    for row in res:
       element =  row.asdict()
       response.append([element['nomGare'].toPython(),element['lat'].toPython(),element['long'].toPython()])
    for elt in response:
        euclid.append(sqrt( (latt - elt[1])**2 + (long - elt[2])**2))
    for i in range(len(euclid)):
        response[i].append(euclid[i])
    response = sorted(response, key=itemgetter(3))
    
    ToHtml = ""
    ToJs = ""

    response = response[:10]
    for i in range(len(response)):
        ToHtml += "<li><b>" + str(response[i][0]) + "</b> — lat: "  + str(response[i][1]) + "; lon: " + str(response[i][2]) + " — " + str(round(response[i][3]*111, 2)) + " km</li>"
        ToJs += "L.marker([" + str(response[i][1]) + ", " + str(response[i][2]) + "]).addTo(mymap).bindPopup('TGV station<br><b>" + str(response[i][0]) + "</b><br>lat. " + str(response[i][1]) + "; lon: " + str(response[i][2]) + "<br>" + str(round(response[i][3]*111, 2)) + " km away');\n"

    return (ToHtml, ToJs)

## Api Flask

In [2]:
from flask import Flask, request, render_template, redirect, url_for, flash, jsonify

import pickle
import json

import rdflib
from math import *
from rdflib import Namespace
import sys
from operator import itemgetter

In [7]:
app = Flask(__name__, template_folder='templates', static_url_path="/static", static_folder='static')

@app.route('/')
def home():
    return render_template("main.html")

@app.route('/result/', methods = ['POST'])
def predictSite():
    # Gathering input data
    latitude = float(request.form['latitude'])
    longitude = float(request.form['longitude'])
    departement = float(request.form['longitude'])
    type_request = request.form['type_query']
    data = [longitude, latitude, type_request]
    type_text = ''
    if type_request == 'tgv_station':
        type_text = 'TGV stations'
    elif type_request == 'university':
        type_text = 'Universities'
    elif type_request == 'both':
        type_text = 'TGV stations and Universities'
    # Calculating by position
    output = getTrainStationByPosition(latitude, longitude)
    return render_template("result.html", output_text=output[0], output_map=output[1], latitude=latitude, longitude=longitude, type_text=type_text)

if __name__ == '__main__':
    app.run(debug = False, host='127.0.0.1')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Mar/2021 14:01:52] "GET / HTTP/1.1" 200 -


PARAM


127.0.0.1 - - [26/Mar/2021 14:01:59] "POST /result/ HTTP/1.1" 200 -
